## Converting STREUSLE Confidence Scores

1. Load all.csv for STREUSLE dev.
2. Change all predicted and ground truth labels to UPOS.
3. Sum relevant confidence scores for new tagset.

In [1]:
# Notebook starts in notebooks folder. Change working directory back to streusle-tagger
%cd ../../../

C:\Michael\dev\streusle-tagger


In [2]:
import pickle

import pandas as pd

In [3]:
dataset = "validation"
streusle_original = pd.read_csv(f"calibration/confidence_scores/{dataset}/all.csv")
streusle_original.head()

,Unnamed: 0,Tokens,Ground Truth,Ground Truth Indexes,Predicted Tags,Predicted Tag Indexes,0,1,2,3,...,586,587,588,589,590,591,592,593,594,595
0,0,Buyer,B-N-n.PERSON,43,O-N-n.PERSON,10,0.000001,1.497282e-05,0.000183,1.315677e-03,...,6.012938e-10,4.828001e-11,2.089973e-09,9.331104e-10,4.117286e-10,1.000886e-10,1.133099e-10,2.731512e-09,6.284988e-10,4.375937e-10
1,1,Beware,I~-V-v.cognition,153,O-V-v.cognition,11,0.000017,7.991283e-05,0.170023,1.093147e-02,...,9.941703e-10,2.139050e-09,2.355739e-09,1.557473e-09,2.996394e-09,1.545388e-09,1.800883e-09,1.086297e-09,2.177137e-09,2.956377e-09
2,2,!!,O-PUNCT,0,O-PUNCT,0,0.999569,1.234659e-06,0.000059,4.653772e-07,...,1.471625e-09,4.450228e-09,6.058480e-10,4.611273e-10,2.255150e-09,2.189517e-09,3.528514e-09,6.002591e-10,1.565019e-09,3.984057e-09
3,0,Rusted,B-V.VPC.semi-v.stative,208,B-ADJ,36,0.000039,6.551610e-08,0.000952,9.642400e-01,...,4.134680e-09,2.032986e-10,5.867963e-09,2.577644e-09,5.865367e-09,5.432755e-10,2.877291e-10,6.196528e-09,6.202440e-09,1.136954e-08
4,1,out,I_,2,I_,2,0.000194,3.034305e-06,0.850971,1.103025e-01,...,4.497888e-09,3.596770e-09,5.126570e-09,9.747019e-09,1.062827e-08,4.760440e-09,3.918880e-09,5.534059e-09,5.810539e-09,1.790509e-08


In [4]:
streusle_consolidated_path = f"calibration/confidence_scores/{dataset}/all_consolidated.csv"

with open("calibration/consolidated_labels.pickle", "rb") as f:
    # Map of each new consolidated label to the corresponding indexes of original labels
    new_labels = pickle.load(f)

streusle_consolidated_path = f"calibration/confidence_scores/{dataset}/all_consolidated_no_lexcat.csv"

with open("calibration/consolidated_labels_no_lexcat.pickle", "rb") as f:
    # Map of each new consolidated label to the corresponding indexes of original labels
    new_labels = pickle.load(f)

In [5]:
streusle_consolidated_columns = ["Token Index", "Tokens", "Predicted Tag", "Predicted Index", "Ground", "Ground Index"] + list(new_labels.keys())
streusle_consolidated = pd.DataFrame(columns=streusle_consolidated_columns)

streusle_consolidated["Token Index"] = streusle_original["Unnamed: 0"]
streusle_consolidated["Tokens"] = streusle_original["Tokens"]

In [6]:
old_index_to_new_label = {}
for k, v in new_labels.items():
    for num in v:
        old_index_to_new_label[num] = k

In [7]:
new_labels_list = list(new_labels)
new_label_to_new_index = dict(zip(new_labels_list, list(range(len(new_labels_list)))))

# Get predicted labels and indexes using consolidated labelset
predicted_labels_consolidated = []
predicted_indexes_consolidated = []

for index in streusle_original["Predicted Tag Indexes"]:
    new_label = old_index_to_new_label[index]
    predicted_labels_consolidated.append(new_label)
    new_index = new_label_to_new_index[new_label]
    predicted_indexes_consolidated.append(new_index)

In [8]:
# Get ground labels and indexes using consolidated labelset
ground_labels_consolidated = []
ground_indexes_consolidated = []

for index in streusle_original["Ground Truth Indexes"]:
    new_label = old_index_to_new_label[index]
    ground_labels_consolidated.append(new_label)
    new_index = new_label_to_new_index[new_label]
    ground_indexes_consolidated.append(new_index)

In [9]:
streusle_consolidated["Predicted Tag"] = predicted_labels_consolidated
streusle_consolidated["Predicted Index"] = predicted_indexes_consolidated
streusle_consolidated["Ground"] = ground_labels_consolidated
streusle_consolidated["Ground Index"] = ground_indexes_consolidated

In [10]:
def sum_scores(new_label, row_index):
    score_sum = 0
    
    for i in new_labels[new_label]:
        score_sum += streusle_original[str(i)][row_index]
        
    return score_sum

In [11]:
score_column_names = list(new_labels.keys())

In [12]:
score_columns = {}
for c in score_column_names:
    score_columns[c] = []
for i, row in streusle_consolidated.iterrows():
    if i % 1000 == 0:
        print(i)
    for c in score_column_names:
        score_columns[c].append(sum_scores(c, i))

0
1000
2000
3000
4000
5000


In [13]:
for c in score_column_names:
    streusle_consolidated[c] = score_columns[c]

In [14]:
streusle_consolidated.head()

,Token Index,Tokens,Predicted Tag,Predicted Index,Ground,Ground Index,O,I,O-v.stative,O-n.group,...,B-??,B-n.plant,o-v.communication,o-v.emotion,b-n.substance,B-n.other,b-v.possession,b-n.quantity,o-n.naturalobject,b-n.location
0,0,Buyer,O-n.person,4,B-n.person,25,0.001562,0.001576,2.423879e-06,0.002828,...,1.266303e-07,8.075447e-09,1.428265e-08,7.513762e-09,1.579287e-08,3.635731e-10,7.732890e-08,6.687372e-11,6.935068e-10,1.133099e-10
1,1,Beware,O-v.cognition,5,I,1,0.011869,0.173678,9.995479e-05,0.000252,...,6.512586e-09,1.857292e-08,1.610332e-08,1.134699e-07,1.466249e-08,8.261513e-09,2.302723e-08,3.137226e-08,1.348522e-08,1.800883e-09
2,2,!!,O,0,O,0,0.999871,0.000073,7.523751e-07,0.000002,...,4.890231e-09,4.156305e-09,1.588622e-09,7.504609e-09,4.571012e-09,9.174098e-09,2.289066e-08,1.914221e-08,5.819978e-09,3.528514e-09
3,0,Rusted,B,23,B-v.stative,41,0.964459,0.001137,9.587081e-03,0.000025,...,7.338792e-10,3.568005e-08,1.756588e-08,1.122264e-09,9.428814e-08,3.735441e-10,3.484682e-07,3.216591e-10,4.824630e-08,2.877291e-10
4,1,out,I,1,I,1,0.122043,0.852345,1.718153e-04,0.000020,...,8.319566e-08,2.395785e-07,1.927984e-08,2.087815e-08,2.744351e-07,1.680764e-09,3.198602e-08,5.296054e-07,2.165121e-07,3.918880e-09


In [15]:
streusle_consolidated.to_csv(streusle_consolidated_path, index=False)